In [22]:
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
from IPython.display import display
from scipy.stats import multinomial, shapiro, anderson

In [2]:
reports = []
for fp in Path('limited_slope/results').glob('*.json'):
    with open(fp, 'r') as fh:
        reports.append(json.load(fh))

In [51]:
tags = {f'{x} db': 0 for x in range(12, 36, 6)}
stats = {'total': tags.copy()}
for report in reports:
    headphone = report['name']
    for result in report['testResults']:
        sample = result['name']
        for option in result['options']:
            if sample not in stats:
                stats[sample] = tags.copy()
            tag = option['name'][-6:-1]
            if tag not in stats[sample]:
                stats[sample][tag] = 0
            stats[sample][tag] += option['count']
            stats['total'][tag] += option['count']
for sample, count_dict in stats.items():
    counts = [int(x) for x in count_dict.values()]
    n = np.sum(counts)
    rv = multinomial(n, [1/len(count_dict) for _ in range(len(count_dict))])
    choices = np.concatenate([[float(v[:2])] * c for v, c in count_dict.items()])
    anders_s, anders_crit, _ = anderson(choices)
    shapiro_w, shapiro_p = shapiro(choices)
    
    count_dict['sum'] = n
    #count_dict['ShapiroW'] = shapiro_w
    count_dict['Shapiro-Wilk P'] = shapiro_p
    #count_dict['Anderson'] = anders_s
    #count_dict['Anderson-5%'] = anders_crit[2]
    #count_dict['p'] = rv.pmf(counts)

In [52]:
df = pd.DataFrame.from_dict(stats, orient='index')
display(df)

,12 db,18 db,24 db,30 db,sum,Shapiro-Wilk P
total,73,68,45,54,240,7.055360e-15
Blink 182 - All The Small Things (intro),12,8,4,6,30,9.572620e-05
Eagles - Hotel California (intro),9,6,8,7,30,5.414024e-04
Holly Cole Trio - I Can See Clearly Now (vocals start),10,7,4,9,30,1.102735e-04
Jennifer Warnes - Bird on a Wire (vocals start),13,6,4,7,30,4.096646e-05
Katie Melua - Shy Boy (vocals start),8,10,5,7,30,5.845715e-04
Michael Jackson - Billie Jean (intro),2,16,7,5,30,1.834393e-04
Steely Dan - Cousin Dupree (vocals start),7,8,7,8,30,9.254733e-04
Wintersun - Sons of Winter and Stars (chorus 1),12,7,6,5,30,1.850714e-04
